In [ ]:
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow # for image display
from skimage import io
from PIL import Image 
import matplotlib.pylab as plt
from google.colab import files
from imutils import contours

In [ ]:
print_in_func = True

In [ ]:
#TO PRINT IMAGES TO THE DRIVE

def drive_print(image, name):
  filename = "drive/My Drive/ELEC4130/test/"+name+".JPG"
  check = cv2.imwrite(filename, image)
  print(check)

In [ ]:
#CONVERT TO B&W

def bwconvert(image, name):
  imagename = name+"_bw"
  image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  
  if print_in_func:
    drive_print(image_grey, imagename)

  return image_grey

In [ ]:
#SIMPLE THRESHOLDING

def simp_th(image, name):
  for i in [50, 60, 70]:
    imagename = name+"_simp_th_"+str(i)
    ret, image_simple_th = cv2.threshold(image, i, 255, cv2.THRESH_BINARY)

    if print_in_func:
      drive_print(image_simple_th, imagename)

In [ ]:
#ADAPTIVE THRESHOLDING (GAUSSIAN)

def adap_th_gaus(image, name):
  image_adaptive_th = 0
  for constant in [15]:
    for block_size in [199]:
      imagename = name+"_adap_th_"+str(block_size)+"_"+str(constant)
      image_adaptive_th = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, constant)
      if print_in_func:
        drive_print(image_adaptive_th, imagename)
  return image_adaptive_th

In [ ]:
#OTSU THRESHOLDING

def otsu_th(image, name):
  for i in [70]:
    imagename = name+"_otsu_th_"+str(i)
    ret, image_simple_th = cv2.threshold(image, i, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if print_in_func:
      drive_print(image_simple_th, imagename)

In [ ]:
#DILATION

def dilate_image(image, name):
  imagename = name+"_dil"
  rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
  dilation = cv2.dilate(image, rect_kernel, iterations = 1)
  if print_in_func:
    drive_print(dilation, imagename)
  return dilation
  

In [ ]:
#CONTOUR SORTER AND AREA SIFTER

def line_avg(vector_list):
  sum = 0
  for vector in vector_list:
    sum = sum + vector[1]
  return (sum/len(vector_list))

def sort_contour(contours):

  box_list = []
  sorted_list = []

  avg_height = 0
  avg_width = 0
  length = 0

  for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if x==0 and y==0:
      continue
    vector = [x, y, w, h]
    avg_height = avg_height+h
    avg_width = avg_width+w
    length = length + 1
    box_list.append(vector)

  avg_height = avg_height/length
  avg_width = avg_width/length

  print("Average Height: ",avg_height)
  print("Average Width: ",avg_width)
  print("Length: ",length)

  for vector in box_list:
    add_check = False
    for num, line in enumerate(sorted_list):
      x_avg = line_avg(line)
      if abs(vector[1]-x_avg)<(1.1*avg_height):
        sorted_list[num].append(vector)
        add_check = True
        break

    if add_check == False:
      new_line = []
      sorted_list.append(new_line)
      length = len(sorted_list)
      sorted_list[(length-1)].append(vector)

  for num,line in enumerate(sorted_list):
    print("\nLine ",str(num))
    for vector in line:
      print(vector)

  return sorted_list
    


In [ ]:
#IMPORT THE IMAGES
# 5828, 5829, 5830, 5831, 5832, 5833,
# 5837, 5842, 5856, 5862, 5863, 5865,
# 5866, 5867, 5868, 5869, 5870, 5871,
# 5872, 5873, 5874, 5875, 5876, 5877,
# 5878, 5879, 5880, 5881, 5882, 5883,
# 5884, 5885, 5887, 5888, 5889, 5890, 
for i in [5833]:
  pathname = "drive/My Drive/ELEC4130/raw/D85_"+str(i)+".JPG"
  imagename = "D85_"+str(i)
  image = cv2.imread(pathname)

  image_grey = bwconvert(image, imagename)

  image_adaptive_th = adap_th_gaus(image_grey, imagename)

  image_dil = dilate_image(image_adaptive_th, imagename)

  contours, hierarchy = cv2.findContours(image_dil, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

  sorted_list = sort_contour(contours)

  counter = 50

  for num,line in enumerate(sorted_list):
    for vector in line:
      x = vector[0]
      y = vector[1]
      w = vector[2]
      h = vector[3]

      rect = cv2.rectangle(image_dil, (x, y), (x + w, y + h), (0, 255, 0), 2)

    tempname = imagename + "_contour_line_"+str(num)
    drive_print(image_dil,tempname)

    counter = counter - 1
    if counter<0:
      break


Streaming output truncated to the last 5000 lines.
[1698, 2558, 18, 24]
[2835, 2556, 34, 39]
[2716, 2556, 33, 38]
[2680, 2556, 31, 38]
[2666, 2556, 9, 38]
[1625, 2557, 18, 9]
[1587, 2557, 16, 24]
[3572, 2555, 3, 3]
[2556, 2555, 31, 38]
[2542, 2555, 9, 37]
[2451, 2555, 51, 37]
[2347, 2555, 9, 37]
[1520, 2556, 17, 9]
[1481, 2556, 18, 24]
[1444, 2556, 18, 25]
[1408, 2556, 18, 8]
[2399, 2554, 31, 38]
[2360, 2554, 34, 38]
[2314, 2554, 9, 37]
[2279, 2554, 30, 38]
[2242, 2554, 33, 38]
[1308, 2555, 19, 25]
[1183, 2555, 18, 8]
[2206, 2553, 33, 51]
[2149, 2553, 34, 38]
[2115, 2553, 31, 38]
[2102, 2553, 9, 37]
[2050, 2553, 31, 38]
[2012, 2553, 35, 37]
[1992, 2553, 16, 51]
[1076, 2554, 18, 24]
[1007, 2554, 18, 24]
[17, 2553, 3, 3]
[3505, 2552, 3, 3]
[1925, 2552, 30, 38]
[809, 2553, 18, 24]
[3433, 2551, 3, 3]
[3269, 2551, 3, 3]
[1890, 2551, 30, 39]
[1835, 2551, 32, 39]
[1782, 2551, 31, 39]
[1745, 2551, 35, 38]
[717, 2552, 18, 9]
[1691, 2550, 32, 39]
[1655, 2550, 31, 38]
[1616, 2550, 34, 38]
[1511, 